In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from sklearn.utils import all_estimators
from time import time
import scipy.stats as stats
from sklearn.utils.fixes import loguniform

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.datasets import load_digits
# Random Forests
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

df = pd.read_csv(r'data/data_processed.csv')
df_cat = pd.read_csv(r'data/data_categorical.csv')

df_good = pd.concat([df_cat, df["MonthlyIncome"], df["BusinessTravel"], df["StockOptionLevel"], df["DistanceFromHome"]], axis=1)

# print(df_good)
# copy attrition and drop from dataframe
JAtt = df["Attrition"].values

df_good = df_good.drop(["Unnamed: 0"], axis=1)
df_good = df_good.drop(['Attrition'], axis=1)
# print(df_good)

# splitting new df
df_train = df_good.iloc[:1200,:]
df_val = df_good.iloc[1200:,:]
# splitting Jatt into new sizes
JAtt_train = JAtt[:1200]
JAtt_val = JAtt[1200:]

from sklearn.ensemble import AdaBoostClassifier


maxiter = np.ceil(10**6 / 1200)

# clf = AdaBoostClassifier(n_estimators=300, random_state=0, learning_rate=1)



In [2]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# df_train = scaler.fit_transform(df_train)

In [3]:

clf = RandomForestClassifier()

clf.fit(df_train, JAtt_train)

y_pred = clf.predict(df_val)

from sklearn import metrics

print(metrics.classification_report(JAtt_val, y_pred))
print(metrics.accuracy_score(JAtt_val, y_pred))


              precision    recall  f1-score   support

           0       0.85      0.99      0.91       221
           1       0.82      0.18      0.30        49

    accuracy                           0.84       270
   macro avg       0.83      0.59      0.61       270
weighted avg       0.84      0.84      0.80       270

0.8444444444444444


In [4]:
# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results["rank_test_score"] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print(
                "Mean validation score: {0:.3f} (std: {1:.3f})".format(
                    results["mean_test_score"][candidate],
                    results["std_test_score"][candidate],
                )
            )
            print("Parameters: {0}".format(results["params"][candidate]))
            print("")

In [8]:
# specify parameters and distributions to sample from
param_dist = {
    'bootstrap': [True, False],
    'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
 }

In [9]:
# run randomized search
n_iter_search = 1500
random_search = RandomizedSearchCV(
    clf, param_distributions=param_dist, n_iter=n_iter_search
)

In [10]:
start = time()
random_search.fit(df_train, JAtt_train)
print(
    "RandomizedSearchCV took %.2f seconds for %d candidates parameter settings."
    % ((time() - start), n_iter_search)
)
report(random_search.cv_results_)

# # use a full grid over all parameters
# param_grid = {
#     "average": [True, False],
#     "l1_ratio": np.linspace(0, 1, num=10),
#     "alpha": np.power(10, np.arange(-2, 1, dtype=float)),
# }

# # run grid search
# grid_search = GridSearchCV(clf, param_grid=param_grid)
# start = time()
# grid_search.fit(df_train, JAtt_train)

# print(
#     "GridSearchCV took %.2f seconds for %d candidate parameter settings."
#     % (time() - start, len(grid_search.cv_results_["params"]))
# )
# report(grid_search.cv_results_)

c:\Users\siman\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\Users\siman\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\Users\siman\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicit

RandomizedSearchCV took 15741.08 seconds for 1500 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.869 (std: 0.019)
Parameters: {'n_estimators': 400, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 90, 'bootstrap': False}

Model with rank: 2
Mean validation score: 0.868 (std: 0.018)
Parameters: {'n_estimators': 600, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': False}

Model with rank: 2
Mean validation score: 0.868 (std: 0.015)
Parameters: {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 90, 'bootstrap': False}

